# **Telco Customer Churn**

This notebook works with a dataset of a fictional telco company that provides home phone and internet services. It indicates which customers have left (churned), stayed or signed up for their service in the last month, and several features about them.

The original dataset is provided by IBM Cognos Analytics, and it consist in five .xlsx files (demographics, population, location, services and status). Each table is described below.

## **Demographics**

- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Gender: The customer’s gender: Male, Female.
- Age: The customer’s current age, in years, at the time the fiscal quarter ended.
- Under 30: Indicates if the customer is younger than 30: Yes, No.
- Senior Citizen: Indicates if the customer is 65 or older: Yes, No.
- Married: Indicates if the customer is married: Yes, No.
- Dependents: Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc.
- Number of Dependents: Indicates the number of dependents that live with the customer.

## **Location**

- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Country: The country of the customer’s primary residence.
- State: The state of the customer’s primary residence.
- City: The city of the customer’s primary residence.
- Zip Code: The zip code of the customer’s primary residence.
- Lat Long: The combined latitude and longitude of the customer’s primary residence.
- Latitude: The latitude of the customer’s primary residence.
- Longitude: The longitude of the customer’s primary residence.

## **Population**

- ID: A unique ID that identifies each row.
- Zip Code: The zip code of the customer’s primary residence.
- Population: A current population estimate for the entire Zip Code area.

## **Services**

- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Quarter: The fiscal quarter that the data has been derived from (e.g. Q3).
- Referred a Friend: Indicates if the customer has ever referred a friend or family member to this company: Yes, No.
- Number of Referrals: Indicates the number of referrals to date that the customer has made.
- Tenure in Months: Indicates the total amount of months that the customer has been with the company by the end of the quarter specified above.
- Offer: Identifies the last marketing offer that the customer accepted, if applicable. Values include None, Offer A, Offer B, Offer C, Offer D, and Offer E.
- Phone Service: Indicates if the customer subscribes to home phone service with the company: Yes, No.
- Avg Monthly Long Distance Charges: Indicates the customer’s average long distance charges, calculated to the end of the quarter specified above.
- Multiple Lines: Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No.
- Internet Service: Indicates if the customer subscribes to Internet service with the company: Yes, No.
- Internet Type: Indicates which type of Internet service the customer suscribes to: None, DSL, Fiber Optic, Cable.
- Avg Monthly GB Download: Indicates the customer’s average download volume in gigabytes, calculated to the end of the quarter specified above.
- Online Security: Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No
- Online Backup: Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No.
- Device Protection Plan: Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No.
- Premium Tech Support: Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No.
- Streaming TV: Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No. The company does not charge an additional fee for this service.
- Streaming Movies: Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No. The company does not charge an additional fee for this service.
- Streaming Music: Indicates if the customer uses their Internet service to stream music from a third party provider: Yes, No. The company does not charge an additional fee for this service.
- Unlimited Data: Indicates if the customer has paid an additional monthly fee to have unlimited data downloads/uploads: Yes, No.
- Contract: Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.
- Paperless Billing: Indicates if the customer has chosen paperless billing: Yes, No.
- Payment Method: Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check.
- Monthly Charge: Indicates the customer’s current total monthly charge for all their services from the company.
- Total Charges: Indicates the customer’s total charges, calculated to the end of the quarter specified above.
- Total Refunds: Indicates the customer’s total refunds, calculated to the end of the quarter specified above.
- Total Extra Data Charges: Indicates the customer’s total charges for extra data downloads above those specified in their plan, by the end of the quarter specified above.
- Total Long Distance Charges: Indicates the customer’s total charges for long distance above those specified in their plan, by the end of the quarter specified above.
- Total Revenue: Indicates the total revenue the company has obtained from the customer, by the end of the quarter specified above.

## **Status**

- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Quarter: The fiscal quarter that the data has been derived from (e.g. Q3).
- Satisfaction Score: A customer’s overall satisfaction rating of the company from 1 (Very Unsatisfied) to 5 (Very Satisfied).
- Customer Status: Indicates the status of the customer at the end of the quarter: Churned, Stayed, or Joined
- Churn Label: Yes = the customer left the company this quarter. No = the customer remained with the company. Directly related to Churn Value.
- Churn Value: 1 = the customer left the company this quarter. 0 = the customer remained with the company. Directly related to Churn Label.
- Churn Score: A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler. The model incorporates multiple factors known to cause churn. The higher the score, the more likely the customer will churn.
- CLTV: Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data. The higher the value, the more valuable the customer. High value customers should be monitored for churn.
- Churn Category: A high-level category for the customer’s reason for churning: Attitude, Competitor, Dissatisfaction, Other, Price. When they leave the company, all customers are asked about their reasons for leaving. Directly related to Churn Reason.
- Churn Reason: A customer’s specific reason for leaving the company. Directly related to Churn Category.

# **Libraries and Functions**

In [ ]:
# Libraries, modules and settings

# !pip install plotly==5.6.0
# !pip install prince==0.7.1

from itertools import combinations

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('notebook')
import plotly.express as px

from prince import MCA

# If running in a Windows 64-bit machine,
# run the lines below to accelerate scikit-learn packages

# from sklearnex import patch_sklearn
# patch_sklearn()

In [ ]:
# Functions


def df_colnames_norm(df):

    '''
    Normalize DataFrame's column names
    (lower strings and no spaces).
    '''

    df.columns = df.columns.str.replace(' ', '_').str.lower()


def df_dropcols_singlevalue(df):

    '''
    Takes a DataFrame and drop columns with only one distinct value.
    '''

    for c in df.columns:
        if df[c].nunique() == 1:
            df.drop(columns=c, inplace=True)


def df_profile(df):

    '''
    Returns a DataFrame with data type, null values, zero values and cardinality.
    '''

    dict_df = {'column': [], 'dtype': [], 'null_n': [], 'null_%': [], 
               'zeros_n': [], 'zeros_%': [], 'unique_n': [], 'unique_%': []}

    for c in df.columns:
        dict_df['column'].append(c)
        dict_df['dtype'].append(df[c].dtype)
        dict_df['null_n'].append(df[c].isna().sum())
        dict_df['null_%'].append(
            '{:.2%}'.format(df[c].isna().sum() / len(df))
            )
        dict_df['zeros_n'].append((df[c]==0).sum())
        dict_df['zeros_%'].append(
            '{:.2%}'.format((df[c]==0).sum() / len(df))
            )
        dict_df['unique_n'].append(df[c].nunique())
        dict_df['unique_%'].append(
            '{:.2%}'.format(df[c].nunique() / len(df))
            )
    
    dict_df = pd.DataFrame(dict_df).set_index('column')
    
    return dict_df


def stacked_corr(corr):

    '''
    Returns a stacked matrix that shows correlation
    between each pair of features row by row (decreasingly in absolute value).
    '''

    stacked_corr = corr.stack().reset_index()
    stacked_corr.columns = ['feature_1', 'feature_2', 'r']
    stacked_corr['features'] = pd.Series(
        list(zip(stacked_corr['feature_1'], stacked_corr['feature_2']))
        )
    
    comb_features = list(combinations(corr.columns, 2))
    
    stacked_corr = stacked_corr.query('features in @comb_features')
    stacked_corr = stacked_corr[['feature_1', 'feature_2', 'r']]
    stacked_corr.loc[:, 'abs_r'] = np.abs(stacked_corr['r'])    
    stacked_corr = stacked_corr.sort_values('abs_r', ascending=False)
    
    return(stacked_corr)


# **Data Reading**

In [ ]:
# DataFrames

df_demographics = pd.read_excel(r'./../data/Telco_customer_churn_demographics.xlsx')
df_location = pd.read_excel(r'./../data/Telco_customer_churn_location.xlsx')
df_population = pd.read_excel(r'./../data/Telco_customer_churn_population.xlsx')
df_services = pd.read_excel(r'./../data/Telco_customer_churn_services.xlsx')
df_status = pd.read_excel(r'./../data/Telco_customer_churn_status.xlsx')

# If running in Colab, load the files and then run:

# df_demographics = pd.read_excel('Telco_customer_churn_demographics.xlsx')
# df_location = pd.read_excel('Telco_customer_churn_location.xlsx')
# df_population = pd.read_excel('Telco_customer_churn_population.xlsx')
# df_services = pd.read_excel('Telco_customer_churn_services.xlsx')
# df_status = pd.read_excel('Telco_customer_churn_status.xlsx')

In order to straightforwardly apply attributes, methods and functions over all DataFrames by iteration, two lists are created: the first containes the DataFrames themselves, and the second contains DataFrames' names.

In [ ]:
# Lists of DataFrames (and DataFrames' names) to iterate over

dfs = [df_demographics, df_location, df_population, df_services, df_status]

dfs_names = ['df_demographics', 'df_location', 
             'df_population', 'df_services', 'df_status']

To avoid problems later, all column names are normalized: spaces are replaced with '_', and all strings are converted to lowercase.

In [ ]:
# Column names normalization over DataFrames

for d in dfs:
    df_colnames_norm(d)

# **Data Wrangling**

## **DataFrames: overview**

As starting point, DataFrames are inspected through some of their basic attributes and methods.

In [ ]:
# DataFrames: shape, columns and info

for i, d in enumerate(dfs):
    print(dfs_names[i], ':\n')
    print('Shape:', d.shape, '\n')
    print('Columns:', d.columns, '\n')
    print('Info:\n')
    print(d.info())
    print('\n--------------------------------------------------------------------------------------------------\n')

All DataFrames (except df_population) have at least two data types (object and integer). Also, all but one of them (again df_population) have same length.

In order to see how the DataFrames look like, a little sample of each one is displayed.

In [ ]:
# DataFrames: samples

dfs_list = []

for i in range(len(dfs)):
    dfs_list.append(dfs_names[i] + ':')
    dfs_list.append('')
    dfs_list.append(dfs[i].sample(5))
    dfs_list.append('')    
    dfs_list.append('----------------------------------------------------------------------------------------------------------------')
    dfs_list.append('')

# Function that allows to display multiple DataFrames in a single cell (takes as argument the previous list(unpacked)), and showing all columns

with pd.option_context('display.max_columns', max(c.shape[1] for c in dfs)):
    display(*dfs_list)

## **Columns: deeper inspection**

Going deeper, it's interesting to count how many null, zero and distinct values are there in each column.

In [ ]:
# DataFrames' columns: cardinality and nulls

# Dictionary of DataFrames, each of one references to cardinality and nulls in original DataFrames' columns

dfs_dict={}

for i, d in enumerate(dfs):
    dfs_dict[dfs_names[i]] = df_profile(d)

# List with names and DataFrames

dfs_list_2 = []

for i in range(len(dfs)):
    dfs_list_2.append(dfs_names[i] + ':')
    dfs_list_2.append('')
    dfs_list_2.append(dfs_dict[dfs_names[i]])
    dfs_list_2.append('')
    dfs_list_2.append('----------------------------------------------------------------------------------------------------------------')
    dfs_list_2.append('')

# Function that allows to display multiple DataFrames in a single cell (takes as argument the previous list(unpacked))

display(*dfs_list_2)

Mostly, columns with object data types have low cardinality and columns with numeric data types have high cardinality, which is the expected pattern. Also, there are some columns with high proportion of zero values and a few ones with null values.

## **Columns with only one distinct value**

Looking at the tables above, it can be seen that some columns have only one distinct value.

In [ ]:
# Distinct values for columns that have only one

for i, d in enumerate(dfs):
    print(dfs_names[i], ':\n')
    for c in d.columns:
        if d[c].nunique() == 1:
            print(d[c].value_counts(), '\n')
    print('----------------------------\n')

Data is about customers living in the United States, in the state of California, and in quarter 3. After taking note of this, the columns can be dropped without any lost of information.

Column 'count' is used in reporting/dashboarding tools to sum up the number of customers in a filtered set, and it can be dropped too.

In [ ]:
for d in dfs:
  df_dropcols_singlevalue(d)

## **Columns with same name among DataFrames**

At this point, it can be noted that there are some columns with the same name among DataFrames. Checking it is a good step, since if these columns also contain the same information, they could be used to merge the DataFrames.

In [ ]:
# Columns with same name in more than one DataFrame

for i, d in enumerate(dfs):
    for j, f in enumerate(dfs[: i] + dfs[i + 1: ]):
        for c in d.columns:
            if c in f.columns:
                print("Column '", c, "' is in both", dfs_names[i], 'and', 
                      (dfs_names[: i] + dfs_names[i + 1: ])[j], 'DataFrames')

- All DataFrames with the same lenght have the common column 'customer_id', which has all distinct values. If data is the same among DataFrames, it means that they can all be merged with one-to-one relations. One easy way to check it is through a set operation.

In [ ]:
dfs_customers = [df_demographics, df_location, df_services, df_status]
dfs_customers_names = ['df_demographics', 'df_location', 'df_services', 'df_status']

for i, d in enumerate(dfs_customers):
  for j, f in enumerate(dfs_customers[: i] + dfs_customers[i + 1: ]):
    print("'customer_id' values are the same in both", dfs_customers_names[i], 
          'and', (dfs_customers_names[: i] + dfs_customers_names[i + 1: ])[j], 
          '?', set(d['customer_id']) == set(f['customer_id']))


> Each DataFrame contains different aspects about the same customers.

- Column 'zip_code' is in both df_population (with all distinct values) and df_location (with not all distinct values). If their values are common (or at least, all values in df_location are included in df_population), these two DataFrames can be merged with a one-to-many relation, so that for each customer, it can be known the population size in his/her district.

In [ ]:
print("'zip_code' values are the same in both df_location and population?", 
      set(df_location['zip_code']) == set(df_population['zip_code']))
print("How many 'zip_code' values in df_population are not included in df_location?",
      len(set(df_population['zip_code']) - set(df_location['zip_code'])))
print("How many 'zip_code' values in df_location are not included in df_population?", 
      len(set(df_location['zip_code']) - set(df_population['zip_code'])))

There are a 45 districts without customers, but all districts with at least one customer have their population info.

## **Null values**

In df_status, there are two columns with null values, 'churn_reason' and 'churn_category', but they could be not missing values.

In [ ]:
print("'churn_label' for null values in 'churn_reason'\n\n", 
      df_status[df_status['churn_reason'].isna()]['churn_label'].value_counts())
print('\n------------------------------------------\n')
print("'churn_label' for not null values in 'churn_reason'\n\n", 
      df_status[df_status['churn_reason'].notna()]['churn_label'].value_counts())
print('\n------------------------------------------\n')
print("'churn_label' for null values in 'churn_category'\n\n", 
      df_status[df_status['churn_category'].isna()]['churn_label'].value_counts())
print('\n------------------------------------------\n')
print("'churn_label' for not null values in 'churn_category'\n\n", 
      df_status[df_status['churn_category'].notna()]['churn_label'].value_counts())

All null values belong to customers that didn't churn, so obviously there's not a churn reason.

# **Exploratory Data Analysis (EDA)**

The EDA will be performed taking the following approach:
- First, each DataFrame will be analyzed individually, in order to gain some insights about different aspects of the customers and population in California.
- Then, all DataFrames will be merged, in order to gain new insights from the available data.

## **Status**

It's convenient to start the analysis with df_status, since it includes the target, this is, whether the customer churnes or not. In turn, this allows to do very useful analytics and calculate some important KPIs.

### **Target and KPIs**

The target is reflected in two columns, 'churn_label' and 'churn_value', and they should be consistent.

In [ ]:
sns.histplot(data=df_status, x='churn_value', discrete=True, 
             hue='churn_label', hue_order=['Yes', 'No'], palette='Set1')

plt.suptitle("df_status: bivariate distribution between 'churn_label' and 'churn_value'")

style=dict(ha='center', va='center', size=14)

# Percentages inside the bars

plt.text(0, (len(df_status) - df_status['churn_value'].sum()) / 2, 
         '{:.2%}'.format((len(df_status) - df_status['churn_value'].sum()) / len(df_status)), 
         **style)
plt.text(1, df_status['churn_value'].sum() / 2, 
         '{:.2%}'.format(df_status['churn_value'].sum() / len(df_status)), 
         **style)

plt.show()

Both columns show the same data in a different way. It's important to note that the percentages displayed in the plot cannot be considered as KPIs, since first it's necessary to substract from the denominator the number of customers that joined in the current quarter, to get the number of customers at the start of the period.

Conveniently, this information is provided by 'customer_status', which makes possible to do all kind of analytics about customers evolution, churning and retention. As starting point, it allows to know the number of customers at the start and at the end of the period.

In [ ]:
print('Number of customers at the start of the quarter:', 
      (df_status['customer_status']!='Joined').sum())
print('Number of customers at the end of the quarter:', 
      (df_status['customer_status']!='Churned').sum())

The company has lost customers respect to the previous period, so the next question is how many customers have been lost.

In [ ]:
print('Number of customers that the company has lost respect to the previous quarter:', 
      (df_status['customer_status']!='Joined').sum() - 
      (df_status['customer_status']!='Churned').sum())
print('Ratio between customers at the end and at the start of the quarter: {:.2%}'.format(
    (df_status['customer_status']!='Churned').sum() / 
    (df_status['customer_status']!='Joined').sum()
    ))

As said before, it's possible from this data to compute some important KPIs, but keeping in mind that ratios must be calculated dividing by the number of customers at the start of the quarter.

$ChurnRate = \frac {NCustomersChurned} {NcustsomersStart}$

$RetentionRate = \frac {NCustomersStayed} {NCustomersStart}$

$ActivationRate = \frac {NCustomersJoined} {NCustormersStart}$

In [ ]:
g = sns.catplot(data=df_status, x='customer_status', col='churn_label', kind='count', 
                order=['Churned', 'Stayed', 'Joined'], col_order=['Yes', 'No'], palette='Set1')

plt.suptitle("df_status: bivariate distribution between 'customer_status' and 'churn_label'", 
             y=1.05)

style=dict(ha='center', va='center', size=12)

# Percentages inside the bars

g.axes[0, 0].text(0, (df_status['customer_status'] == 'Churned').sum() / 2, 
                  'Churn rate:\n{:.2%}'.format(
                      (df_status['customer_status'] == 'Churned').sum() / 
                      (df_status['customer_status'] != 'Joined').sum()
                      ), **style)
g.axes[0, 1].text(1, (df_status['customer_status'] == 'Stayed').sum() / 2, 
                  'Retention rate:\n{:.2%}'.format(
                      (df_status['customer_status'] == 'Stayed').sum() / 
                      (df_status['customer_status'] != 'Joined').sum()
                      ), **style)
g.axes[0, 1].text(2, (df_status['customer_status'] == 'Joined').sum() / 2, 
                  'Activation rate:\n{:.2%}'.format(
                      (df_status['customer_status'] == 'Joined').sum() / 
                      (df_status['customer_status'] != 'Joined').sum()
                      ), **style)

plt.show()

Category 'Yes' in 'churn_label' is exactly the same that category 'Churned' in 'customer_status', while category 'No' in 'churn_label' can be viewed with more detail in 'customer_status', among categories 'Joined' and 'Stayed'.

Tipically, the next step would be to make a cohort analysis. However, this is not possible because for doing this, the dataset should contain the customers that joined and churned in previous periods. Anyway, it's still possible to explore more specific KPIs with the available data.

- Average Customers Lifetime: it measures the number of periods that customers stay with the company, in average (assuming a constant churn rate). It can be calculated as the reciprocal of the churn rate.

> $ACLT = \frac 1 {Churn Rate} = \frac {NCustomersStart} {NCustomersChurned}$

> Its importance lies in the fact that, for being profitable, customers must give benefits before this time.

In [ ]:
print('Average Customers Lifetime:', round(
    (df_status['customer_status'] != 'Joined').sum() / 
    (df_status['customer_status'] == 'Churned').sum(), 4
    ), 'quarters.')

> In average, customers must report benefits between months 1 and 10 of their lifetime; otherwise, the company will not cover the acquisition costs associated. Of course, it's incorrect to supose that all customers belong to the same distribution. Previously, it's necessary to make a good segmentation, so that for each cluster, it can be calculated the average lifetime. But this can't be done until merging the DataFrames, since more features are needed.

- Future Dimension of the Business: this concept arises from applying the average lifetime to the customers that joined in the current period. In other words, it estimates, from historical data, the total number of periods that new customers will stay in the company.

> $FDB = NCustomersJoined * ACLT = \frac {NCustomersJoined * NCustomersStart} {NCustomersChurned}$



In [ ]:
print('Future Dimension of the Business:', round(
    (df_status['customer_status'] == 'Joined').sum() * 
    (df_status['customer_status'] != 'Joined').sum() / 
    (df_status['customer_status'] == 'Churned').sum(), 2
    ), 'quarters.')

- Increase in the Dimension of the Business: ratio between the Future Dimension of the Business and the number of customers at the start of the period, minus 1. Through some algebraic operations, it's possible to switch its calculus as the ratio between the activation and the churn rate, minus 1.

> $IDB = \frac {FDB} {NCustomersStart} - 1 = \frac {NCustomersJoined} {NCustomersChurned} - 1 = \frac {ActivationRate} {ChurnRate} - 1$

> It indicates, from a customer lifetime approach, if the number of customers is increasing or decreasing.

In [ ]:
print('Increase in Dimension of Business: {:.2%}'.format(
    ((df_status['customer_status'] == 'Joined').sum() / 
     (df_status['customer_status'] == 'Churned').sum()) - 1
     ))

> The situation of the company in the aspects of retention and customers lifetime (in California), taking in account the available data (only the current quarter) is really awful.



### **Categorical features**

Moving to the next features, 'churn_reason' and 'churn_category' should be consistent.

In [ ]:
pd.crosstab(index=df_status['churn_reason'], columns=df_status['churn_category'])

Clearly, 'churn_category' is a grouped version of 'churn_reason'. It seems some observations in 'Poor expertise of online support' are misclassified, and should belong to 'Dissatisfation' instead of 'Other'.

In [ ]:
df_status.loc[df_status['churn_reason'] == 
              'Poor expertise of online support', 
              'churn_category'] = 'Dissatisfaction'

At this step, in order to avoid the problem of high cardinality in categorical features, 'churn_reason' won't be considered for the EDA. Anyway, this column won't be dropped, since its data could be very insightfull in some aspects about retention, especially after merging DataFrames. For example, studying if there's a relationship between charges and churning because of 'Price' reasons, or between services an churning because of 'Dissatisfaction' reasons, etc.



It's useful to create a list with 'churn_category' and 'customer_status', to iterate over them when studying bivariate distribution with numerical features. Also, data types in all these columns are converted to Categorical.

In [ ]:
cols_cat_status = ['customer_status', 'churn_category']

for c in cols_cat_status:
  df_status[c] = pd.Categorical(df_status[c])

df_status['churn_reason'] = pd.Categorical(df_status['churn_reason'])
df_status['churn_label'] = pd.Categorical(df_status['churn_label'])

Finally, 'customer_status' and 'churn_category' should be consistent.

In [ ]:
g = sns.catplot(data=df_status, x='churn_category', col='customer_status', kind='count',
                order=df_status['churn_category'].dropna().unique().sort_values(), 
                col_order=['Churned', 'Stayed', 'Joined'], palette='Set2')

plt.suptitle("df_status: bivariate distribution between 'churn_category' and 'customer_status'", 
             y=1.05)

# 45° rotation of all xticks in the catplot

for j in range(df_status['customer_status'].nunique()):
  g.axes[0, j].tick_params(axis='x', rotation=45)

# Sorting unique values ensures that each percentage is in inside the correct bar
# Nulls in 'churn_category' are dropped

for i, c in enumerate(df_status['churn_category'].dropna().unique().sort_values()):
  g.axes[0, 0].text(range(5)[i], (df_status['churn_category']==c).sum() / 2, 
                    '{:.2%}'.format((df_status['churn_category']==c).sum() / 
                                    df_status['churn_value'].sum()), 
                    ha='center', va='center', size=12)

plt.show()

Almost half of churns are due to 'Competitor' reasons, followed by 'Dissatisfaction' and 'Attitude'. 'Price' and 'Other' at the bottom of the rank.

### **Numerical features**

First of all, it's convenient to look at the statistical measures of each feature.

In [ ]:
df_status.describe().T.round(3)

All numerical features (but not the target) have their mean very close to the median, which suggests they are quite simetric.

Naturally, the next step is to analyze correlation among them. Since the target is dichotomic, it's more appropriate to use a non parametric method for correlation.

In [ ]:
corr_status_k = df_status.corr(method='kendall')

sns.heatmap(corr_status_k.round(3), annot=True, vmin=-1, vmax=1, 
            cmap=sns.diverging_palette(220, 20, as_cmap=True))

plt.suptitle('df_status: Kendall correlation', y=1.05)

plt.xticks(rotation=45)

plt.show()

As expected, the target is correlated positively with 'churn_score' and negatively with 'satisfaction_score'. In the other hand, the correlation between the target and 'cltv' (customer lifetime value) is very near to zero. This is surprising, since it implies at least one of the following situations:
- The company isn't good at holding valuable customers.
- The company estimates very poorly the true CLTV.

Similarly to categorical features, it's convenient to create a list with numerical features, which provides an easy way to study bivariate distribution with categorical features.

In [ ]:
cols_num_status = list(df_status.select_dtypes(include=np.number).columns)
cols_num_status.remove('churn_value')

### **Bivariate distribution (categorical vs numerical)**

The following grid shows:
- In the first row, distribution of categorical features.
- In the first column, distribution of numerical features.
- In the other axes, bivariate distribution of categorical features against numerical ones.

In [ ]:
fig, axes = plt.subplots(nrows=len(cols_num_status) + 1, 
                         ncols=len(cols_cat_status) + 1, 
                         figsize=((len(cols_cat_status) + 1) * 6, 
                                  (len(cols_num_status) + 1) * 5), 
                         sharex='col', sharey='row')

plt.suptitle('df_status: bivariate distributions between categorical and numerical features', 
             y=0.92)

# Lists of values for plots arguments

discrete = [True, False, False]
order = [['Churned', 'Stayed', 'Joined'], 
         df_status['churn_category'].dropna().unique().sort_values()]
palette = ['Set1', 'Set2']

# Histplots for numerical features in first column

for i, n in enumerate(cols_num_status):
  sns.histplot(data=df_status, y=n, ax=axes[i + 1, 0], 
               discrete=discrete[i])
  
# Countplots for categorical features in first row

for j, c in enumerate(cols_cat_status):
  sns.countplot(data=df_status, x=c, ax=axes[0, j + 1], 
                order=order[j], palette=palette[j])

# Boxplots for bivariates distribution between numerical and categorical features

for i, n in enumerate(cols_num_status):
  for j, c in enumerate(cols_cat_status):
    sns.boxplot(data=df_status, x=c, y=n, ax=axes[i + 1, j + 1], 
                order=order[j], palette=palette[j])

# 45° rotation of categorical xticks

for k in range(2):
  axes[len(cols_num_status), k + 1].set_xticklabels(
      labels=axes[len(cols_num_status), k + 1].get_xticklabels(), rotation=45)

# Percentages inside bars for discrete histplot

for s in df_status['satisfaction_score'].unique():
  axes[1, 0].text((df_status['satisfaction_score'] == s).sum() / 2, s, 
                  '{:.2%}'.format((df_status['satisfaction_score'] == s).sum() / 
                                  len(df_status)), 
                  size=14, ha='center', va='center')

# Since it's not used, first ax is removed

axes[0, 0].remove()

plt.show()

Since 'customer_status' is stretchly related to the target, their bivariate distribution against numerical features are similar (category 'Churned' associated to high values in 'churn_score' and low values in 'satisfaction_score', and independent with 'cltv'). Particularly, all customers with 'satisfaction_score' lesser than 3 churned, and no customer with 'satisfaction_score' greater than 3 churned. On the other hand, there's quite independence in bivariate distribution between 'churn_category' and numerical features.

From the definition of the tables and features at the start of the notebook, it follows that 'churn_score' is an output of a previous model built by the company. It's interesting to inspect how good it is at predicting churning by itself.

In [ ]:
sns.swarmplot(data=df_status.sample(250), x='customer_status', y='churn_score', 
              hue='customer_status', hue_order=['Churned', 'Stayed', 'Joined'], 
              size=3, palette='Set1')

plt.suptitle("df_status: swarmplot 'churn_score' against 'customer_status'")

plt.show()

The predictions are very good, except for 'churn_score' lower than 80 and higher than 65. Maybe, other features can help to be more accurate.

In [ ]:
sns.pairplot(data=df_status.drop(columns='churn_value').sample(150), hue='customer_status', 
             hue_order=['Churned', 'Stayed', 'Joined'], palette='Set1')

plt.suptitle('df_status: pairplot', y=1.05)

plt.show()

It comes clear from the plot that just by taking in account 'satisfaction_score', the predictions could be significantly more accurate. Probably, this feature wasn't taken in account in the previous model, and a better one can be designed.

## **Demographics**

Now it's turn to analyze df_demographics. In this case, there are some features expressed in both categorical and numerical format. The approach for these groups of features will be to inspect them together, in order to perform a descriptive analysis and look if they are consistent, all at the same time.

### **Dependents**

In this case there's a categorical feature which indicates whether the customer has dependents or not, and a numerical feature which indicates how many these dependents are.

In [ ]:
sns.histplot(data=df_demographics, x='number_of_dependents', 
             hue='dependents', discrete=True)

plt.suptitle("df_demographics: bivariate distribution between 'dependents' and 'number_of_dependents'")

# Percentages inside bars for significant values

for i in range(4):
  plt.text(i, (df_demographics['number_of_dependents']==i).sum() / 2, 
           '{:.2%}'.format((df_demographics['number_of_dependents']==i).sum() / 
                           len(df_demographics)), 
           ha='center', va='center', size=8)

plt.show()

The classification is right. A few more than three quarters of the customers doesn't have dependents. For the rest, the number of customers with 1, 2 and 3 dependents is quite similar. Finally, there could be a few customers with more than 3 dependents.

In [ ]:
df_demographics.query('number_of_dependents > 3')['number_of_dependents'].value_counts().sort_index().to_frame()

If considered as features of a model which predicts whether the customer churns or not, it seems convenient to keep 'dependents' (encoded as dummy) and drop 'number_of_dependents', because of the strong right asymmetry in the last one.

### **Age**

In this case there's one numerical feature 'age', and two categorical features, 'under_30' and 'senior_citizen', which indicate the age range of customers.

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(8, 8), sharex='col')

plt.suptitle("df_demographics: bivariate distributions between 'age' and categorical ages")

# Histplots for 'age' and categorical features related

sns.histplot(data=df_demographics, x='age', ax=axes[0], 
             hue='under_30', binwidth=5, binrange=(15, 84))
sns.histplot(data=df_demographics, x='age', ax=axes[1], 
             hue='senior_citizen', binwidth=5, binrange=(15, 84))

# Percentages inside histplots for each category of both categorical features

for i, c in enumerate(['under_30', 'senior_citizen']):
  for ca in df_demographics[c].unique():
    axes[i].text(df_demographics[df_demographics[c] == ca]['age'].mean(), 200, 
                 '{:.2%}'.format((df_demographics[c] == ca).sum() / 
                                 len(df_demographics)), 
                 ha='center', size=12)

plt.show()

A fifth of the customers are under 30, and a few more than 15 % of them are over 65. It can be noted that these categorical features are consistent but overlapped, so it's convenient to join them.

In [ ]:
cond_list = [(df_demographics['under_30'] == 'Yes'), 
             (df_demographics['under_30'] == 'No') & 
             (df_demographics['senior_citizen'] == 'No'), 
             (df_demographics['senior_citizen'] == 'Yes')]

choice_list = ['under_30', 'adult', 'senior_citizen']

df_demographics['age_category'] = np.select(cond_list, choice_list)

Once this is done, overlapped features can be dropped. 

In [ ]:
df_demographics = df_demographics.drop(columns=['under_30', 'senior_citizen'])

If considered as features of models which predict whether the customer churns or not, at least one of them should be dropped, since they represent the same information in a different way. It could be the case that for some kind of models, 'age' performed better, and for other kind of models, 'age_category' performed better. This should be explored through some cross-validation method.

Again, it's useful to create a list with the relevant categorical features (the previous ones, but also 'gender' and 'marriage), to iterate over them when studying bivariate distribution with numerical features, and to convert data types in these columns to Categorical.

In [ ]:
cols_cat_demographics = ['gender', 'married', 'dependents', 'age_category']

for c in cols_cat_demographics[:3]:
  df_demographics[c] = pd.Categorical(df_demographics[c])

df_demographics['age_category'] = pd.Categorical(df_demographics['age_category'],
                                                 categories=['under_30', 'adult', 'senior_citizen'],
                                                 ordered=True)

### **Numerical features**

As usual, statistical measures of each feature are displayed.

In [ ]:
df_demographics.describe().T.round(3)

The fact that 'age' 's mean is extremely close to its median is consistent with the symmetry observed in its histogram. In the same way, the fact that more than three quarters of the customers have no dependents is consistent with the strong right asymmetry observed in 'number_of_dependents' ' histogram.

In order to look for relations between the numerical features, it's now more appropiate to use Pearson correlation, since there are not dichotomic ones.

In [ ]:
corr_demographics_p = df_demographics.corr()

sns.heatmap(corr_demographics_p.round(3), annot=True, vmin=-1, vmax=1, 
            cmap=sns.diverging_palette(220, 20, as_cmap=True))

plt.suptitle('df_demographics: Pearson correlation', y=1.05)

plt.show()

Counterintuitively, 'age' and 'number_of_dependents' are linearly uncorrelated, but it still could be the case of a non linear relation between them. The best way to discover it is through a scatter plot.

In [ ]:
sns.scatterplot(data=df_demographics.sample(150), x='age', y='number_of_dependents')

plt.suptitle("df_demographics: bivariate distribution between 'age' and 'number_of_dependents'")

plt.show()

It seems that 'age' for customers with at least one dependent is more constrained to working age. Anyway, the relation is not strong enough.

Again, a list with numerical features is created.

In [ ]:
cols_num_demographics = list(df_demographics.select_dtypes(include=np.number).columns)

### **Bivariate distribution (numerical vs categorical)**

The following grid shows:
- In the first row, distribution of numerical features.
- In the first column, distribution of categorical features.
- In the other axes, bivariate distribution of numerical features against categorical ones.
- There are quite few customers with 4 or more dependents. For 'number_of_dependents' histogram and box plots, these cases are dropped in order to make the plots more interpletable.

In [ ]:
fig, axes = plt.subplots(nrows=len(cols_cat_demographics) + 1, 
                         ncols=len(cols_num_demographics) + 1, 
                         figsize=((len(cols_num_demographics) + 1) * 5, 
                                  (len(cols_cat_demographics) + 1) * 4), 
                         sharex='col', sharey='row')

plt.suptitle('df_demographics: bivariate distributions between categorical and numerical features', 
             y=0.92)

# Lists of values for plots arguments 

data = [df_demographics, df_demographics.query('number_of_dependents <= 3')]
discrete = [False, True]
binwidth = [10, None]
binrange = [(15, 84), None]

# Countplots for categorical features in first column

for i, c in enumerate(cols_cat_demographics):
  sns.countplot(data=df_demographics, y=c, ax=axes[i + 1, 0])

# Histplots for numerical features in first row

for j, n in enumerate(cols_num_demographics):
  sns.histplot(data=data[j], x=n, ax=axes[0, j + 1], 
               discrete=discrete[j], binwidth=binwidth[j], binrange=binrange[j])

# Boxplots for bivariates distribution between numerical and categorical features

for i, c in enumerate(cols_cat_demographics):
  for j, n in enumerate(cols_num_demographics):
    sns.boxplot(data=data[j], x=n, y=c, ax=axes[i + 1, j + 1])

# Since it's not used, first ax is removed

axes[0, 0].remove()

plt.show()

From this grid:
- Box plots 'age' vs 'age_category' and 'number_of_dependents' vs 'dependents' are trivial.
- Numerical feature 'age' is quite uncorrelated against both 'gender', 'married', and slightly related to 'dependents' (particularly, higher ages tend to have no dependents).
- Numerical feature 'number_of_dependents' is quite uncorrelated against 'gender', and correlated against 'married' (married customers tend to have more dependents) and 'age_category' (customers over 65 tend to have no dependents, and there's no difference between adults and under 30 customers).

## **Services**

Similarly to df_demographics, there's a feature expressed in both categorical and numerical format. The approach will be the same, this is, inspect them together in order to perform a descriptive analysis and look if they are consistent, all at the same time.

### **Referrals**

In this case there's a categorical feature which indicates whether the customer has referred at least one friend or not, and a numerical feature which indicates how many these referrals are.

In [ ]:
sns.histplot(data=df_services, x='number_of_referrals', 
             discrete=True, hue='referred_a_friend')

plt.suptitle("df_services: bivariate distribution between 'referrals' and 'number_of_referrals'")

# Percentages inside bars for significant values

for i in range(2):
  plt.text(i, (df_services['number_of_referrals']==i).sum() / 2, 
           '{:.2%}'.format((df_services['number_of_referrals']==i).sum() / 
                           len(df_services)), 
           rotation='vertical', ha='center', va='center', size=12)

plt.show()

The classification is right. A few more than half of the customers didn't referred anyone, and 15 % of them referred only one friend. About 30 % referred from 2 to 10 friends, and there could be a few of them who referred more than 10.

In [ ]:
df_services.query('number_of_referrals > 10')['number_of_referrals'].value_counts().sort_index().to_frame()

If considered as features of a model which predicts whether the customer churns or not, it seems convenient to keep 'referred_a_friend' (encoded as dummy) and drop 'number_of_referrals', because of the strong right asymmetry in the last one.

### **Numerical features**

As usual, the starting point is to look at the statistical measures of each features.

In [ ]:
df_services.describe().T.round(3)

Since there are so many numerical features, it's convenient to look at their statistical distribution closer.

In [ ]:
cols_num_services = list(df_services.select_dtypes(include=np.number).columns)

fig, axes = plt.subplots(nrows=(len(cols_num_services) + 4) // 5 * 2, ncols=5,
                         figsize=(18, ((len(cols_num_services) + 4) // 5) * 8))

plt.suptitle('df_services: distribution of numerical features', y=0.92)

# List of values for discrete argument in histplot

discrete = [True, False, False, False, False, False, False, False, False, False]

# Histplots and boxplots in axes layout

for i, n in enumerate(cols_num_services):
  sns.histplot(data=df_services, x=n, ax=axes[2 * (i // 5), i % 5], 
               kde=True, discrete=discrete[i])
  sns.boxplot(data=df_services, x=n, ax=axes[2 * (i // 5) + 1, i % 5])

plt.show()

It can be noted that:
- Feature 'avg_monthly_long_distance_charges' has a near uniform distribution (except for zero values).
- Features 'monthly_charge' and 'tenure_in_months' seems to have bimodal distribution.
- Features 'total_charges', 'total_long_distance_charges', 'total_revenue' and 'avg_monthly_gb_download' have right asymmetry. If considered as features of a model that predicts whether the customer churns or not, it could be necessary (depending on the type of model) treat this asymmetry. This could be done (for features with no zero values) taking the log instead.
- Feature 'number_of_referrals' has a strong right asymmetry. As said before, if considered as a feature of a model which predicts whether the customer churns or not, the better choice is probably to keep only 'referred_a_friend'.
- Features 'total_refunds' and 'total_extra_data_charges' have an extreme right asymmetry. It's convenient to dichotomize them, not only for predictive but also for descriptive purposes.

In [ ]:
# Dichotomizing 'total_refunds'

cond_list = [df_services['total_refunds'] == 0, 
             df_services['total_refunds'] > 0]

choice_list = ['No', 'Yes']

df_services['refunds'] = np.select(cond_list, choice_list)

df_services['refunds'] = pd.Categorical(df_services['refunds'])

In [ ]:
# Dichotomizing 'total_extra_data_charges'

cond_list = [df_services['total_extra_data_charges'] == 0, 
             df_services['total_extra_data_charges'] > 0]

choice_list = ['No', 'Yes']

df_services['extra_data_charges'] = np.select(cond_list, choice_list)

df_services['extra_data_charges'] = pd.Categorical(df_services['extra_data_charges'])

Again, since there are not numerical dichotomic features ('refunds' and 'extra_data_charges' will be treated with the categorical ones), the best choice for look at the correlation is Pearson method.

In [ ]:
corr_services_p = df_services.corr()

plt.figure(figsize=(15, 12))

sns.heatmap(corr_services_p.round(3), annot=True, vmin=-1, vmax=1, 
            cmap=sns.diverging_palette(220,20,as_cmap=True))

plt.suptitle('df_services: Pearson correlation', y=0.92)

plt.xticks(rotation=45)

plt.show()

Although several features have strong correlation, there's an important issue here: some of them could be due to accounting relationships. For example, for each customer, the total revenue plus the total refunds given back to him must be equal to the total charges for all services and extras.

In [ ]:
print("'total_revenue' + 'total_refunds' - ('total_charges' + 'total_long_distance_charges' + 'total_extra_data_charges')\n")

print('Maximum difference:', 
      (df_services['total_revenue'] + df_services['total_refunds'] - 
       (df_services['total_charges'] + 
        df_services['total_long_distance_charges'] + 
        df_services['total_extra_data_charges'])).max())
print('Minimum difference:', 
      (df_services['total_revenue'] + df_services['total_refunds'] - 
       (df_services['total_charges'] + 
        df_services['total_long_distance_charges'] + 
        df_services['total_extra_data_charges'])).min())

Similarly, for each customer, the tenure multiplied by the average monthly charges must equal the total charges.

In [ ]:
print("'total_long_distance_charges' - ('tenure_in_months' * 'avg_monthly_long_distance_charges')\n")

print('Maximum difference:', 
      (df_services['total_long_distance_charges'] - 
       (df_services['tenure_in_months'] * 
        df_services['avg_monthly_long_distance_charges'])).max())
print('Minimum difference:', 
      (df_services['total_long_distance_charges'] - 
       (df_services['tenure_in_months'] * 
        df_services['avg_monthly_long_distance_charges'])).min())

In [ ]:
pd.concat((df_services['tenure_in_months'] * df_services['monthly_charge'], 
           df_services['total_charges']), axis=1).rename(
               columns={0: 'tenure_in_months*monthly_charge'}).corr()

Once those correlations due to accounting relations are taken off, no significant ones remain.

Finally, 'total_refunds' and 'total_extra_data_charges' can be removed, and categorical 'refunds' and 'extra_data_charges' used instead.

In [ ]:
df_services.drop(columns=['total_refunds', 'total_extra_data_charges'], inplace=True)

cols_num_services = list(df_services.select_dtypes(include=np.number).columns)

### **Categorical features**

Once more, it's useful to list the categorical features, to iterate over them when studying their univariate distribution and their bivariate distribution with numerical features. But this time there's a particular situation: several features are subclassifications of other categorical features, since they indicate if the customer has acquired a sub-service that is provided in the context of a broader service. To make the analysis as clear as possible, three list are created: the first lists the broad services and other features, the second lists phone sub-services, and the third lists internet sub-services.

Again, data types in these columns are converted to Categorical.

In [ ]:
cols_cat_services = ['referred_a_friend', 'offer', 'phone_service', 'internet_service', 
                     'contract', 'paperless_billing', 'payment_method', 'refunds']

cols_cat_phone_services = ['multiple_lines']

cols_cat_internet_services = ['internet_type', 'online_security', 'online_backup', 
                              'device_protection_plan', 'premium_tech_support', 
                              'streaming_tv', 'streaming_movies', 'streaming_music', 
                              'unlimited_data', 'extra_data_charges']

cols_cat_services_subservices = cols_cat_services + cols_cat_phone_services + cols_cat_internet_services

for c in (cols_cat_services_subservices):
  df_services[c] = pd.Categorical(df_services[c])

As starting point, broad services and other features are plotted ('referrals' is omitted, since it was analyzed before).

In [ ]:
fig, axes = plt.subplots(nrows=(len(cols_cat_services[1:]) + 2) // 3, ncols=3,
                         figsize=(18, ((len(cols_cat_services[1:]) + 2) // 3) * 4))

plt.suptitle('df_services: distribution of categorical features', y=0.92)

# Countplots in axes layout

for i, c in enumerate(cols_cat_services[1:]):
  sns.countplot(data=df_services, x=c, ax=axes[i // 3, i % 3], 
                order=df_services[c].unique().sort_values())

# Sorting unique values ensures that each percentage is in inside the correct bar

  for j, ca in enumerate(df_services[c].unique().sort_values()):
    axes[i // 3, i % 3].text(
        j, 
        (df_services[c] == ca).sum() / 2, 
        '{:.2%}'.format((df_services[c] == ca).sum() / len(df_services)), 
        ha='center', va='center'
        )

# Axes that are not used are removed

axes[2, 1].remove()
axes[2, 2].remove()

plt.show()

From this grid:
- A few more than half of the customer didn't accepted any offer.
- Nearly a tenth of the customers doesn't have phone service.
- A few more than a fifth of the customers doesn't have internet service.
- A few more than half of the customers have a month to month contract, and nearly a half of them have a one year or a two years contract.
- Three fifths of the customers have paperless billing.
- A few more than half of the customers pay with bank withdrawal, two fifths of them do it with credit card, and a few of them do it with mailed check.
- Nearly 7.5 % of the customers received some refunds from the company.

#### **Phone sub-services**

When analyzing a sub-service, an important point is how the categories in it are related with the categories in its broader service. In the case of 'phone_service', there's only one sub-service related, 'multiple_lines'.

In [ ]:
g = sns.catplot(data=df_services, x='multiple_lines', col='phone_service', kind='count', 
                aspect=1.25, order=df_services['multiple_lines'].unique().sort_values(),
                col_order=df_services['phone_service'].unique().sort_values())

plt.suptitle("df_services: bivariate distribution between 'multiple_lines' and 'phone_service'", 
             y=1.05)

style = dict(ha='center', size=12)

# Percentages are taken to customers that have (or not) 'phone_service'

for i, ca in enumerate(df_services['phone_service'].unique().sort_values()):
  for j, subca in enumerate(df_services['multiple_lines'].unique().sort_values()):
    if ((df_services['phone_service'] == ca) & (df_services['multiple_lines'] == subca)).sum() > 0: 
      g.axes[0, i].text(j, ((df_services['phone_service'] == ca) & (df_services['multiple_lines'] == subca)).sum() / 3 * 2, 
                        '% all customers: {:.2%}'.format(
                            ((df_services['phone_service'] == ca) & (df_services['multiple_lines'] == subca)).sum() / 
                            len(df_services)), **style)
      g.axes[0, i].text(j, ((df_services['phone_service'] == ca) & (df_services['multiple_lines'] == subca)).sum() / 3, 
                        '% phone service {}: {:.2%}'.format(ca, 
                            ((df_services['phone_service'] == ca) & (df_services['multiple_lines'] == subca)).sum() / 
                            (df_services['phone_service'] == ca).sum()), **style)
      
plt.show()

It's clear that those customers that don't have phone service, either could have multiple lines. But an issue arises because the same category is used for customers that have phone service but don't have multiple line. Due to this, it's essential to analyze both 'phone_service' and 'multiple_lines' together.

For customers that have phone service, a few more than half of them don't have multiple lines.

#### **Internet sub-services**

In the case of 'internet_service', there are several sub-services related.

In [ ]:
style = dict(ha='center', size=12)

for c in cols_cat_internet_services:
  g = sns.catplot(data=df_services, x=c, col='internet_service', kind='count', 
                  aspect=1.5, order=df_services[c].unique().sort_values(), 
                  col_order=df_services['internet_service'].unique().sort_values())
  
  plt.suptitle("df_services: bivariate distribution between '" + c + "' and 'internet_service'", 
               y=1.05)

  # Percentages are taken to customers that have (or not) 'internet_service'
  
  for i, ca in enumerate(df_services['internet_service'].unique().sort_values()):
    for j, subca in enumerate(df_services[c].unique().sort_values()):
      if ((df_services['internet_service'] == ca) & (df_services[c] == subca)).sum() > 0:
        g.axes[0, i].text(j, ((df_services['internet_service'] == ca) & (df_services[c] == subca)).sum() / 3 * 2, 
                          '% all cust: {:.2%}'.format(
                              ((df_services['internet_service'] == ca) & (df_services[c] == subca)).sum() / 
                              len(df_services)), **style)
        g.axes[0, i].text(j, ((df_services['internet_service'] == ca) & (df_services[c] == subca)).sum() / 3, 
                          '% int serv {}: {:.2%}'.format(ca, 
                              ((df_services['internet_service'] == ca) & (df_services[c] == subca)).sum() / 
                              (df_services['internet_service'] == ca).sum()), **style)
        
  plt.show()

As before, it's essential to analyze each sub-service together with 'internet_service'. From the grid, and for customers that have internet service:
- A few more than half of them are connected by Fiber Optic, three tenths of them are connected by DSL and the rest are connected by Cable.
- Most of them have unlimited data.
- Most of them have paid extra data charges.
- Half of them have streaming TV.
- Half of them have streaming movies.
- Nearly half of them have streaming music.
- Nearly half of them have online backup.
- Nearly half of them have device protection plan.
- A few more than a third of them have online security.
- A few more than a third of them have premium tech support.

Going further, it's possible to analyze bivariate distribution between 'internet_type' and each sub-service:

In [ ]:
style = dict(ha='center', size=10)

for c in cols_cat_internet_services[1:]:
  g = sns.catplot(data=df_services, x=c, col='internet_type', kind='count', 
                  aspect=0.75, order=df_services[c].unique().sort_values(), 
                  col_order=df_services['internet_type'].unique().sort_values())
  
  plt.suptitle("df_services: bivariate distribution between '" + c + "' and 'internet_type'", 
               y=1.05)
  
  # Percentages are taken to customers that have each type of internet service
  
  for i, ca in enumerate(df_services['internet_type'].unique().sort_values()):
    for j, subca in enumerate(df_services[c].unique().sort_values()):
      if ((df_services['internet_type'] == ca) & (df_services[c] == subca)).sum() > 0:
        g.axes[0, i].text(j, ((df_services['internet_type'] == ca) & (df_services[c] == subca)).sum() / 3 * 2, 
                          '% all cust: {:.2%}'.format(
                              ((df_services['internet_type'] == ca) & (df_services[c] == subca)).sum() / 
                              len(df_services)), **style)
        g.axes[0, i].text(j, ((df_services['internet_type'] == ca) & (df_services[c] == subca)).sum() / 3, 
                          '% int serv {}: {:.2%}'.format(ca, 
                              ((df_services['internet_type'] == ca) & (df_services[c] == subca)).sum() / 
                              (df_services['internet_type'] == ca).sum()), **style)
        
  plt.show()

From this grid:
- Customers with Cable or DLS internet service are more likely to contract 'online_security' and 'premium_tech_support' than customers with Fiber Optic.
- Customers with Fiber Optic internet service are more likely to use the streaming services (the company does not charge an additional fee for them).
- Sub-services 'online_backup', 'device_protection_plan', 'unlimited_data' and 'extra_data_charges' have the same distribution among the three types of internet service.

It can be noted that 'unlimited_data' and 'extra_data_charges' seem to have almost the same distribution, but some labels are changed.

In [ ]:
pd.crosstab(index=[df_services['unlimited_data'], df_services['extra_data_charges']], 
            columns=df_services['internet_service'])
            

This is because customers who don't have internet service can't be suscribed to unlimited data sub-service, nor pay extra data charges. On the other hand, for customers who have internet service, those who have unlimited data don't pay extra data charges, and those who haven't suscribed this sub-service tend to pay extra data charges. A few ones of them don't have unlimited data and don't pay extra data charges. This could be due to low 'avg_monthly_gb_download'.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 5), sharey='row')

plt.suptitle("df_services: bivariate distribution between 'extra_data_charges' and 'avg_monthly_gb_download' for 'internet_service'=='Yes' and 'unlimited_data'=='No'")

data=df_services.query("internet_service=='Yes' and unlimited_data=='No'")

sns.kdeplot(data=data, y='avg_monthly_gb_download', hue='extra_data_charges', ax=axes[0])
sns.violinplot(data=data, x='extra_data_charges', y='avg_monthly_gb_download', ax=axes[1])

plt.show()

Counterintuitively, the relation is not clear. Maybe these customers have a bonification that's not specified in the dataset. Anyway, since the cases are really few, 'extra_data_charges' gives almost the same information that 'unlimited_data', and can be dropped for further analyses.

In [ ]:
df_services.drop(columns='extra_data_charges', inplace=True)

cols_cat_internet_services = ['internet_type', 'online_security', 'online_backup', 
                              'device_protection_plan', 'premium_tech_support', 
                              'streaming_tv', 'streaming_movies', 'streaming_music', 
                              'unlimited_data']

cols_cat_services_subservices = cols_cat_services + cols_cat_phone_services + cols_cat_internet_services

#### **Multiple Correspondence Analysis**

Although it's interesting to know about the multivariate distribution of all sub-services, it would be very painful to plot each one against each one of them. Nonetheless, there's an multivariate statistical method that allows to see, not without some loss of information, all the categories in a 2D plot. 

In [ ]:
mca_services = MCA(n_components=len(cols_cat_services_subservices))
mca_services = mca_services.fit(df_services[cols_cat_services_subservices])

ax = mca_services.plot_coordinates(
     X = df_services[cols_cat_services_subservices],
     ax=None,
     figsize=(16, 16),
     show_row_points=False,
     row_points_size=7,
     show_row_labels=False,
     show_column_points=True,
     column_points_size=40,
     show_column_labels=True,
     legend_n_cols=1
     )

First of all, it's important to note that only 28 % of the information is captured in this plot (the most inertia is in the horizontal axis)

The closer the categories are in the plot (especially though the horizontal axis), there are more customers that belong to both categories at the same time. It seems that some categories can be grouped in clusters, for example:
- Multiple lines with cable and DSL.
- Streaming services.
- Online backup and device protection plan.
- Online security and premium tech support.
- Internet service and unlimited data.
- Bank withdrawal and paperless billing.
- Credit card and no paperless billing.
- Referred a friend and one year contract.


### **Bivariate distribution (categorical vs numerical)**

The following grid shows:
- In the first row, distribution of categorical features.
- In the first column, distribution of numerical features.
- In the other axes, bivariate distribution of categorical features against numerical ones.

In [ ]:
fig, axes = plt.subplots(nrows=len(cols_num_services) + 1, 
                         ncols=len(cols_cat_services) + 1, 
                         figsize=((len(cols_cat_services) + 1) * 4, 
                                  (len(cols_num_services) + 1) * 4), 
                         sharex='col', sharey='row')

plt.suptitle('df_services: bivariate distributions between categorical and numerical features', 
             y=0.9)

# Lists of values for plots arguments

discrete = [True, False, False, False, False, False, False, False, False, False]
order = [df_services[c].unique().sort_values() for c in cols_cat_services]

# Histplots for numerical features in first column

for i, n in enumerate(cols_num_services):
  sns.histplot(data=df_services, y=n, ax=axes[i + 1, 0], 
               discrete=discrete[i])
  
# Countplots for categorical features in first row

for j, c in enumerate(cols_cat_services):
  sns.countplot(data=df_services, x=c, ax=axes[0, j + 1], 
                order=order[j])

# Boxplots for bivariates distribution between numerical and categorical features

for i, n in enumerate(cols_num_services):
  for j, c in enumerate(cols_cat_services):
    sns.boxplot(data=df_services, x=c, y=n, ax=axes[i + 1, j + 1], 
                order=order[j])

# 45° rotation of categorical xticks

for k in range(len(cols_cat_services)):
  axes[len(cols_num_services), k + 1].set_xticklabels(
      labels=axes[len(cols_num_services), k + 1].get_xticklabels(), rotation=45)

# Since it's not used, first ax is removed

axes[0, 0].remove()

plt.show()

Despite the grid is very hard to analyze because of there are several categorical and numerical features, some interesting insights can be taken:
- Offers A to E are intended for tenure segmented customers. Offer A is intended for high tenure customers and Offer E is intended for low tenure customers. Knowing this, it seems a good choice to dichotomyze 'offer', and just show whether the customer has accepted the offer or not.
- Customers with high tenure tend to have a two years contract, customers with medium tenure tend to have a one year contract, and customers with low tenure tend to have a month to month contract.
- Customers with low tenure tend to have a mailed check payment method.
- Customers with high tenure tend to have referred more friends.
- Customers with high monthly charge tend to have paperless billing and to pay with bank withdrawal.
- Of course, the monthly charge is related to the services that the customer has suscribed.
- Since the tenure and the monthly charge are closely related to total amounts, the relation between the total amounts and features mentioned above are quite similar.
- The average monthly GB download and the average monthly long distance charges have not significant relations with categorical features.

In [ ]:
# Dichotomizing 'offer'

cond_list = [df_services['offer'] == 'None', 
             df_services['offer'] != 'None']

choice_list = ['No', 'Yes']

df_services['offer'] = np.select(cond_list, choice_list)

df_services['offer'] = pd.Categorical(df_services['offer'])

It's known from previous steps than 55 % of customers didn't accept any offer, so it's no necessary to plot the feature again.

## **Population**

This DataFrame doesn't hold any data about customers, but combined with df_location, it can help to understand penetration of the company throughtout the state. As starting point, it's useful to see how population in California is distributed. Note that in this case, population is related to each zip code in the state, but it can also be related to other levels, like cities or counties.

In [ ]:
df_population[['population']].describe().T.round(3)

In [ ]:
sns.histplot(data=df_population, x='population')

plt.suptitle("df_population: distribution of 'population' for districts in California")

plt.show()

As expected, it has right symmetry (there are a few districts with high population, and several ones with lower population).

## **Location**

Previously, it was said that 'df_population' and 'df_location' could be merged with a one-to-many relation, so that for each customer, it can be known the latitude, longitude and population in the district, and also to which city the district belongs to.

In [ ]:
df_loc_pop = pd.merge(df_location, df_population.drop(columns='id'), 
                      how='left', on='zip_code')

df_loc_pop.sample(5)

From the point of view of relational databases design, it would have been more expectable that df_location only informed the zip code to which every singles customer belongs, and another DataFrame informed not only the population in each zip code, but also the name of the city and location. This pattern would prevent from inconsistences that could arise, for example, if in df_location, two customers had the same zip code but different cities (or latitudes, or longitudes).

From an analytical point of view, this design would also allow to study the distribution of population with a geographical dimension, previous to analyze how customers are distributed throughout the state.

### **Districts**

Anyway, a DataFrame like this can be created, through dropping duplicated rows from df_loc_pop. For semantical purposes, the areas to which each zip code belongs will be called districts.

In [ ]:
df_districts = df_loc_pop.drop(
    columns=['customer_id', 'lat_long']
    ).drop_duplicates()

df_districts.rename(columns={'population': 'population_district'}, 
                    inplace=True)

print('Number of rows in df_districts:', len(df_districts), '\n')

print("unique_% for 'zip_code': {:.2%}\n".format(
    df_districts['zip_code'].nunique() / len(df_districts)
    ))

df_districts.sort_values(by='population_district', ascending=False)

The fact that all values in 'zip_code' are distinct means that 'cities', 'latitude' and 'longitude' are consistent respect to 'zip_code', in the sense descripted in the previous point. However, the oppossite isn't true, since big cities involve several zip codes.

In [ ]:
print("unique_% for 'city': {:.2%}".format(
    df_districts['city'].nunique() / len(df_districts)
    ))

Values in column 'city' are not all distinct. It's possible, through groupby and filter operations, to list cities with more than one zip code.

In [ ]:
def filter_multiple_zipcodes(df):

    '''
    Filter function passed through filter method of groupby object.
    For each key value of the groupby object, it returns the rows
    whose values in 'zip_code' are not all distinct.
    '''

    return df['zip_code'].nunique() > 1


print('Number of cities in California with more than one zip code:', 
      len(df_districts.groupby(by='city').filter(
          filter_multiple_zipcodes
          ).groupby(by='city')[['zip_code']].nunique()), '\n\nCities:\n')

df_districts.groupby(by='city').filter(
    filter_multiple_zipcodes
    ).groupby(by='city')[['zip_code']].nunique().sort_values(
        by='zip_code', ascending=False
        ).head(10)

### **Cities**

Keeping this in mind, it seems reasonable to aggregate population for each city. Since the location isn't available for cities, but only for districts, it will be imputed as the mean of 'latitude' and 'longitude'. But first, it's necessary to check there are no cities with repeated names throughout the state. One way to do this is to calculate:
- The difference between maximum and minimum latitude (longitude) as a proxy of the distance between north and south (east and west) limits of the state.
- For each district, the difference between its latitude (longitude) and the mean for the city to which it belongs, and then, look at the resulting distribution.

In [ ]:
print("Difference between maximum and minimum 'latitude':", 
      df_districts['latitude'].max() - df_districts['latitude'].min())
print("Difference between maximum and minimum 'longitude':", 
      df_districts['longitude'].max() - df_districts['longitude'].min())

In [ ]:
df_districts[['city', 'latitude']].groupby(by='city').transform(
    lambda x: np.abs((x - x.mean()))
    ).plot(kind='box')

plt.suptitle("df_districts: distribution of 'latitude' and 'longitude'")

df_districts[['city', 'longitude']].groupby(by='city').transform(
    lambda x: np.abs((x - x.mean()))
    ).plot(kind='box')

plt.show()

There are several outliers, but the strong right asymmetry can be understood thinking in lots of cities with small areas and a few ones with big areas. Compared to distances between limits of the state, even the most extreme values seem acceptable. Thus, the groupby operation makes sense.

In [ ]:
df_cities = df_districts.groupby(by='city').aggregate(
    {'latitude': 'mean', 'longitude': 'mean', 'population_district': 'sum'})

df_cities.rename(columns={'population_district': 'population_city'}, inplace=True)

df_cities.reset_index(inplace=True)

df_cities.sort_values(by='population_city', ascending=False)

Again, it's useful to look at the distribution of cities' population in California.

In [ ]:
df_cities[['population_city']].describe().T.round(3)

In [ ]:
sns.histplot(data=df_cities, x='population_city')

plt.suptitle("df_cities: distribution of 'population' for cities in California")

plt.show()

As expected, it has largely more right asymmetry than districts case. In order to study the geographical distribution of cities' population, it's convenient to transform data taking the log (if not, it turns very difficult to look at the difference between cities with low population).

In [ ]:
df_cities['population_city_log10'] = df_cities['population_city'].apply(
    lambda x: np.log10(x))

sns.histplot(data=df_cities, x='population_city_log10')

plt.suptitle("df_cities: distribution of log10('population') for cities in California")

plt.show()

Finally, it's possible to analyze population for each city in a map. Colors are based on the log transformation, but the true population is informed in the hover data.

In [ ]:
df_cities_sample = df_cities.sample(250)

fig = px.scatter_geo(
    data_frame=df_cities_sample, lat='latitude', lon='longitude', 
    color_continuous_scale="Viridis", color='population_city_log10', 
    hover_name='city', hover_data=['population_city'], 
    title='Geographical distribution of cities and population', 
    template='ggplot2'
    )

fig.update_geos(fitbounds='locations')

fig.show()

There are two big poles, one around Los Ángeles and San Diego, and the other around San José, San Francisco and Sacramento.

### **Counties**

Another way to visualize geographical distribution of population is drawing limits of counties, but this information isn't available in the dataset. Anyway, this can be reached by adding the information from external resources.

In [ ]:
df_zipcodes = pd.read_excel(r'./../data/CA_fipscodes_zipcodes.xlsx', sheet_name=0)
df_fipscodes = pd.read_excel(r'./../data/CA_fipscodes_zipcodes.xlsx', sheet_name=1, 
                            dtype={'FIPS': str})

# If running in Colab, load the files and then run:

# df_zipcodes = pd.read_excel('CA_fipscodes_zipcodes.xlsx', sheet_name=0)
# df_fipscodes = pd.read_excel('CA_fipscodes_zipcodes.xlsx', sheet_name=1, 
#                              dtype={'FIPS': str})

display('df_zipcodes', '', df_zipcodes, '', 
        '------------------------------------------------------------------------------------------------------------------------------',
        '', 'df_fipscodes', '', df_fipscodes)

From this data:
- df_zipcodes allows to know to which county each district belongs to.
- df_fipscodes provides the FIPS code for each county, which will be passed to the plot as argument.

As with the original dataset, it is useful to know hoy many null and distinct values are there in each DataFrame.

In [ ]:
display('df_zipcodes', '', df_profile(df_zipcodes), 
        '------------------------------------------------------------------------------------------------------------------------------',
        '' , 'df_fipscodes', '', df_profile(df_fipscodes))

There are 58 different counties in California. As expected, 'zip_code' , 'FIPS' and 'county' have all distinct values, and 'county' has one null value.

It's necessary to perform some string operationes to make data compliant.

In [ ]:
df_zipcodes['county'] = df_zipcodes['county'].str[:-7]

df_fipscodes['FIPS'] = '0' + df_fipscodes['FIPS']

Previous to perform merge operations, it´s a good step to check, for common columns, if also values are common.

In [ ]:
print("'zip_code' values are the same in both df_zipcodes and df_population?", 
      set(df_population['zip_code']) == set(df_zipcodes['zip_code']))
print("How many 'zip_code' values in df_zipcodes are not included in df_population?", 
      len(set(df_zipcodes['zip_code']) - set(df_population['zip_code'])))
print("How many 'zip_code' values in df_population are not included in df_zipcodes?",
      len(set(df_population['zip_code']) - set(df_zipcodes['zip_code'])), '\n')

print("'county' values are the same in both df_zipcodes and df_fipscodes?", 
      set(df_zipcodes['county']) == set(df_fipscodes['county']))
print("How many 'county' values in df_fipscodes are not included in df_zipcodes?", 
      len(set(df_fipscodes['county']) - set(df_zipcodes['county'])))
print("How many 'county' values in df_zipcodes are not included in df_fipscodes?", 
      len(set(df_zipcodes['county']) - set(df_fipscodes['county'])))
print('Which value is that?', set(df_zipcodes['county']) - set(df_fipscodes['county']))

In [ ]:
print("'zip_code' in df_population for the null value of 'county':\n") 
df_population[df_population['zip_code']==df_zipcodes[df_zipcodes['county'].isna()]['zip_code'].values[0]]

There are several zip codes that are not included in df_population. It will be assumed that missing zip codes are deprecated, or that belong to towns with very few habitants. Contrary, there's no lack of information about counties (the null value for 'county' belongs to a zip code that is not included in df_population), so every zip code in df_population can be related to a county, and thus, to a FIPS code. All this means that the merge operations required can be done without lost of information.

In [ ]:
df_counties = pd.merge(
    pd.merge(
        df_population, df_zipcodes[['zip_code', 'county']], how='left', on='zip_code'
        ), df_fipscodes[['FIPS', 'county']], how='left', on='county').groupby(
            by=['FIPS', 'county'])[['population']].sum()

df_counties.rename(columns={'population': 'population_county'}, inplace=True)

df_counties.reset_index(inplace=True)

df_counties.sort_values(by='population_county', ascending=False)

Again, it's useful to look at the distribution of counties' population in California.

In [ ]:
df_counties[['population_county']].describe().T.round(3)

In [ ]:
sns.histplot(data=df_counties, x='population_county')

plt.suptitle("df_counties: distribution of 'population' for counties in California")

plt.show()

Although it doesn't have as much right asymmetry as cities case, it still could be the case that log transformation improved the study about geographical distribution of counties' population.

In [ ]:
df_counties['population_county_log10'] = df_counties['population_county'].apply(lambda x: np.log10(x))

sns.histplot(data=df_counties, x='population_county_log10')

plt.suptitle("df_counties: distribution of log10('population') for counties in California")

plt.show()

Finally, it's possible to analyze population for each county in a map. Colors are based on the log transformation, but the true population is informed in the hover data.

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

lat_mean = df_districts['latitude'].mean()    # For centering the map
lon_mean = df_districts['longitude'].mean()

fig = px.choropleth(df_counties, 
    geojson=counties, 
    locations='FIPS', 
    color='population_county_log10',
    color_continuous_scale="Viridis",
    hover_name='county',
    hover_data=['population_county'],
    scope="usa",
    center={'lat': lat_mean, 'lon': lon_mean},
    title='Geographical distribution of population among counties', 
    template='ggplot2'
)

fig.show()

Going further, it's possible to know the number of customers for each county. For this purpose:
- First, df_loc_pop is merged with df_cities (on 'city'), df_zicodes (on 'zip_code') and df_county (on 'county'), so that, for each customer, it can be known the county where he or she lives, and the the population in both the city and the county where he or she lives.

In [ ]:
df_loc_pop = pd.merge(
    pd.merge(
        pd.merge(
            df_loc_pop.rename(columns={'population': 'population_district'}).drop(columns='lat_long'), 
            df_cities[['city', 'population_city', 'population_city_log10']], how='left', on='city'), 
             df_zipcodes[['zip_code', 'county']], how='left', on='zip_code'), 
             df_counties, how='left', on='county')

df_loc_pop = df_loc_pop.iloc[:, [0, 3, 4, 2, 1, 9, 8, 5, 6, 7, 10, 11]]

df_loc_pop.sample(5)

- Second, through a groupby operation, it's possible to calculate the aggregation of customers for each county. Despite it could be done just in df_loc_pop, it's more appropriate to it merged it to df_counties again.

In [ ]:
df_counties = pd.merge(df_counties, df_loc_pop.groupby(by='county')[['customer_id']].nunique(), 
                       how='left', on='county').rename(columns={'customer_id': 'customers_county'})

df_counties

It would be expectable that counties with high population also had many customers.

In [ ]:
sns.scatterplot(data=df_counties, x='population_county', y='customers_county')

plt.suptitle("df_counties: relation between 'population_county' and 'customers_county'")

plt.show()

Roughly, the relation keeps, so a geographic plot of the number of customers doesn't make sense, since it would be very similar to the population one. However, it's still interesting to study the penetration of the company in each county. This can be done taking the ratio between these features.

In [ ]:
df_counties['ratio_cust_pop'] = round(df_counties['customers_county'] / 
                                      df_counties['population_county'], 6)

Again, it's useful to look at the distribution of this new feature.

In [ ]:
df_counties[['ratio_cust_pop']].describe().T.round(3)

In [ ]:
sns.histplot(data=df_counties, x='ratio_cust_pop')

plt.suptitle("df_counties: distribution of 'customers_county'/'population_county' in California")

plt.show()

Since it has strong right asymmetry, it's convenient to take the log before plotting.

In [ ]:
df_counties['ratio_cust_pop_log10'] = df_counties['ratio_cust_pop'].apply(lambda x: np.log10(x))

sns.histplot(data=df_counties, x='ratio_cust_pop_log10')

plt.suptitle("df_counties: distribution of log10('customers_county'/'population_county') in California")

plt.show()

In [ ]:
fig = px.choropleth(df_counties, 
    geojson=counties, 
    locations='FIPS', 
    color='ratio_cust_pop_log10',
    color_continuous_scale="Viridis",
    hover_name='county',
    hover_data=['population_county', 'ratio_cust_pop'],
    scope="usa",
    center={'lat': lat_mean, 'lon': lon_mean}, 
    title="Geographical distribution of 'customers_county'/'population_county' among counties", 
    template='ggplot2'
)

fig.show()

As a general pattern, penetration is higher for counties with low population, mainly in the north and east of the state.

## **Merged**

At this point, all DataFrames have been enriched quite enough, and a complete merge operation can be performed, so that for each customer, there´s a complete set of features that describe them and can be used in predictive models. Since all DataFrames have a one to one relation, the same result can be achieved by column concatenation, setting the common column as index.

In [ ]:
df = pd.concat([df_loc_pop.set_index('customer_id'), 
                df_demographics.set_index('customer_id'),
                df_services.set_index('customer_id'), 
                df_status.set_index('customer_id')], 
               axis=1)

df.reset_index(inplace=True)

print('Shape:', df.shape, '\n')

with pd.option_context('display.max_columns', df.shape[1]):
    display(df.sample(5))

### **Monetary KPI: Gross Monthly Recurrent Revenue Churn Rate**

Now that all information is available, it's possible to compute KPIs that involve monetary measures. For example, it's possible to compute the the Gross Monthly Recurrent Revenue Churn Rate, this is, the percentage of income lost due to churn in the period.

$GMRRC = \frac {\sum MonthlyChargeChurners} {\sum MonthlyChargeStart}$

In [ ]:
print('Monetary Recurrent Revenue Churn Rate: {:.2%}'.format(
    (df.query("customer_status=='Churned'")['monthly_charge'].sum() + 
     df.query("customer_status=='Churned'")['avg_monthly_long_distance_charges'].sum()) / 
     (df.query("customer_status!='Joined'")['monthly_charge'].sum() +
      df.query("customer_status!='Joined'")['avg_monthly_long_distance_charges'].sum())
))

It's a little bit higher than Churn rate.

In addition, the Net Monthly Recurrent Revenue Churn Rate measures the income lost due to churn and downgrades in the service, and adjusted by upgrades and service expansions. Since this data isn't provided in the dataset, this KPI can't be computed. As said before (when talking about cohort analysis), having historical data would improve the study.

### **Geographical distribution of KPIs**

Another way the analysis can be improved now is to look at the geographical distribution of KPIs among counties. The first step to do this is to pass functions that calculate the KPIs to a groupby object, with the counties as keys.

In [ ]:
def churn_rate(df):

    '''
    Function passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the rate of customers that churned in the current quarter.
    '''

    df['churn_rate'] = round(
        (df['customer_status']=='Churned').sum() / \
        (df['customer_status']!='Joined').sum(), 6
        )
    
    return df


def mrrchurn_rate(df):

    '''
    Function passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the rate of monthly revenue lost because of churnings in the current quarter.
    '''

    df['mrrchurn_rate'] = round(
        (df.query("customer_status=='Churned'")['monthly_charge'].sum() + \
         df.query("customer_status=='Churned'")['avg_monthly_long_distance_charges'].sum()) / \
         (df.query("customer_status!='Joined'")['monthly_charge'].sum() + \
          df.query("customer_status!='Joined'")['avg_monthly_long_distance_charges'].sum()), 6
          )

    return df


def retention_rate(df):

    '''
    Function passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the rate of customers that stayed in the current quarter.
    '''

    df['retention_rate'] = round(
        (df['customer_status']=='Stayed').sum() / \
        (df['customer_status']!='Joined').sum(), 6
        )
    
    return df


def activation_rate(df):

    '''
    Function passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the rate of customers that joined in the current quarter.
    '''

    df['activation_rate'] = round(
        (df['customer_status']=='Joined').sum() / \
        (df['customer_status']!='Joined').sum(), 6
        )
    
    return df


def fdb(df):

    '''
    Fuction passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the future dimension of the business in the current quarter.
    '''

    df['fdb'] = round(
        (df['customer_status']=='Joined').sum() * \
        (df['customer_status']!='Joined').sum() / \
        (df['customer_status']=='Churned').sum(), 4
        )
    
    return df


def idb(df):

    '''
    Function passed through apply method of groupby object.
    For each key value of the groupby object,
    it returns the increase in future dimension in the current quarter.
    '''

    df['idb'] = round(
        ((df['customer_status']=='Joined').sum() / \
         (df['customer_status']=='Churned').sum()) - 1, 6
         )
    
    return df


Since the result is displayed in df, the second step is to merge it to df_counties.

In [ ]:
df_counties = pd.merge(
    df_counties, df[['customer_id', 'county', 'customer_status', 'monthly_charge', 'avg_monthly_long_distance_charges']].groupby(
        by='county').apply(churn_rate).groupby(by='county').apply(mrrchurn_rate).groupby(by='county').apply(retention_rate).groupby(
            by='county').apply(activation_rate).groupby(by='county').apply(fdb).groupby(by='county').apply(idb).drop(
                columns=['customer_id', 'monthly_charge', 'avg_monthly_long_distance_charges', 'customer_status']).drop_duplicates(), 
                how='left', on='county'
                )

df_counties

It's useful, before plotting, to look at the distribution of KPIs.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))

sns.histplot(data=df_counties, x='churn_rate', ax=axes[0, 0])
sns.histplot(data=df_counties, x='retention_rate', ax=axes[0, 1])
sns.histplot(data=df_counties, x='activation_rate', ax=axes[0, 2])
sns.histplot(data=df_counties, x='mrrchurn_rate', ax=axes[1, 0])
sns.histplot(data=df_counties, x='fdb', ax=axes[1, 1])
sns.histplot(data=df_counties, x='idb', ax=axes[1, 2])

plt.suptitle("Distribution of KPIs in California")

plt.show()

Only 'fdb' has a strong right assymetry. In order to study the geographical distribution of this feature, it's convenient to transform it taking the log.

In [ ]:
df_counties['fdb_log10'] = df_counties['fdb'].apply(lambda x: np.log10(x))

sns.histplot(data=df_counties, x='fdb_log10')

plt.suptitle("df_counties: distribution of log10('fdb') in California")

plt.show()

It would be expectable that 'churn_rate' and 'mrrchurn_rate' were related.

In [ ]:
line = np.linspace(0, pd.concat((df_counties['churn_rate'], df_counties['mrrchurn_rate'])).max(), 100)

sns.scatterplot(data=df_counties, x='churn_rate', y='mrrchurn_rate')
sns.lineplot(x=line, y=line, color='red')

plt.suptitle("df_counties: relation between 'churn_rate' and 'mrrchurn_rate' (and 45° line)")

plt.show()

The relation is very close to be linear, and it's also close to the 45° line, so it's enough to plot only one of them. It can be observed that, in average, 'mrrchurn_rate' tends to be a little higher than 'churn_rate'. This can be understood as customers with higher monthly charges churning a little bit more than those with lower monthly charges.

Since 'churn_rate', 'retention_rate' and 'activation_rate' have perfect multicollinearity, only the first will be plotted. Anyway, all of them will be informed in the hover data.

In [ ]:
fig = px.choropleth(
    df_counties, 
    geojson=counties, 
    locations='FIPS', 
    color='churn_rate',
    color_continuous_scale="Viridis",
    hover_name='county',
    hover_data=['population_county', 'idb', 'fdb', 'mrrchurn_rate', 'activation_rate', 'retention_rate'],
    scope="usa",
    center={'lat': lat_mean, 'lon': lon_mean}, 
    title="Geographical distribution of 'churn_rate' in counties", 
    template='ggplot2'
    )

fig.show()

Some of the counties with high population have also a very high Churn rate. Especially, San Diego has near 3M habitants and the Churn rate over 40 %. Taking back the analysis of churn reasons, 9 of each 20 customers that churned did it because of 'Competitors'. Probably, competence is stronger in high population areas.

In the other hand, the Future Dimension of the Business can be analyzed taking the log.

In [ ]:
fig = px.choropleth(df_counties, 
    geojson=counties, 
    locations='FIPS', 
    color='fdb_log10',
    color_continuous_scale="Viridis",
    hover_name='county',
    hover_data=['population_county', 'churn_rate', 'retention_rate', 'activation_rate', 'idb', 'fdb'],
    scope="usa",
    center={'lat': lat_mean, 'lon': lon_mean}, 
    title="Geographical distribution of 'fdb' in counties", 
    template='ggplot2'
)

fig.show()

Since this KPI takes in account not only ratios but also the number of customers, it's expectable to have some correlation with the population. This is why Los Angeles has the higher FDB, despite the ratios are not so good. It's the state where it's probably that the new customers will significate more future incomes. In the other hand, San Diego does it worst than other counties with less population.

### **Geographical distribution of churning and CLTV**

In order to finish the geographical analysis, it's possible to look at individual distribution of customers. In the following plot, a sample of customers is displayed, and also their status and the estimated Customer Lifetime Value.

In [ ]:
df_sample = df.sample(50)
fig = px.scatter_geo(
    data_frame=df_sample, lat='latitude', lon='longitude', 
    size='cltv', size_max=7, color='customer_status', 
    category_orders={'customer_status': ['Stayed', 'Churned', 'Joined']}, 
    title="Geographical distribution of customers, 'customer_status' and 'cltv'"
    )
fig.update_geos(fitbounds="locations")
fig.show()

As said before, it's very surprising that there's not a strong relation between churning and CLTV, which means that the company isn't good at holding valuable customers (or the company estimates the CLTV very poorly).

## **Conclusion of EDA**

The analysis made before showed that the company has a serious problem in the aspect of retention, and that the number of customers has decreased respect to the start of the quarter. Going further, it was seen that the main cause that churners answer when they are asked is 'Competitors'.

Going further, a geographical analysis showed that some metrics about penetration and retention are especially poor in counties with high population, which has a significant impact in global metrics. Relating this to what was said in the previous paragraph, it can be inferred that the market is much more competitive in big cities and counties, and for the company is very difficult to deal with this fact. It's necessary some more investigation in order to identify the causes of this lack of competitiveness.

Additionally, the company is unable to classify well and retain customers with high Customer Lifetime Value, which negatively affects its long term profits.

There are lots more of small conclusion that can be taken, but since they were described as the EDA advanced, they can be ommitted at this point.

# **Data Curation**

Despite some statistical methods allow to select features in order to reach the best performance of models, the explainability of these decreases as the complexity increases, what turns difficult to quantify the effect of each individual feature for a given prediction. For this reason, as a previous step of predictive analysis, it's useful to study the relation between the target and the features.

## **Stacked correlations and Clustermap**

First of all, it's useful to see how all pair of numerical features (and target) are related.

In [ ]:
corr_p = df.drop(columns=['zip_code', 'latitude', 'longitude']).corr()
stacked_corr(corr_p).query('abs_r > 0.6')

Since the target is dichotomic, it's more appropriate to use a non parametric method for correlation.

In [ ]:
corr_k = df.drop(columns=['zip_code', 'latitude', 'longitude']).corr(method = 'kendall')
stacked_corr(corr_k).query("feature_1=='churn_value' or feature_2=='churn_value'")

The target is highly correlated with 'satisfaction_score' and 'churn_score', but the last one is the output of a previous model, so it won't be considered for a new one. Furthermore, the target has some correlation with 'tenure_in_months'. On the other hand, the correlation is a little bit higher for 'total_charges' that for 'monthly_charge', but since the former is highly correlated to 'tenure_in_months', it should be preferred the latter.

It can be viewed through a clustermap the groups of features that have stronger association.

In [ ]:
sns.clustermap(corr_p, cbar_pos=(-0.05, 0.8, 0.05, 0.18), cmap='coolwarm', center=0)

plt.suptitle('df: clustermap', y=1.05)

plt.show()

As expected, the monetary features are correlated, and the same happens with the population features. Furthermore, 'cltv' is correlated with 'tenure_in_months' and the monetary features, which means that these are probably the features that the company uses for its calculus.

## **Bivariate distribution between numerical features and target**

Going further, it's possible look at the same kind of bivariate plots used in previous sections ir order to get a clearer understanding of the relations between features and the target. For convenience, the first plot includes numerical features only (those ones from df_status aren't taken in account, since their bivariate distribution against the target has been already studied).

In [ ]:
cols_num = list(df.drop(
    columns=['zip_code', 'satisfaction_score', 'churn_value', 'churn_score', 'cltv']
    ).select_dtypes(include=np.number).columns)

fig, axes = plt.subplots(nrows=len(cols_num) + 1, ncols=2, 
                         figsize=(16, (len(cols_num) + 1) * 5), 
                         sharey='row')

plt.suptitle('df_status: bivariate distributions between numerical features and target', 
             y=0.89)

# Countplot for the target in first row

sns.countplot(data=df, x='churn_label', ax=axes[0, 1], 
              order=['Yes', 'No'], palette='Set1')

# Histplots for numerical features in first column
# Violinplots for bivariate distribution in second colum

for i, n in enumerate(cols_num):
  sns.kdeplot(data=df, y=n, ax=axes[i + 1, 0], hue='churn_label', 
              hue_order=['Yes', 'No'], palette='Set1')
  sns.violinplot(data=df, x='churn_label', y=n, ax=axes[i + 1, 1], 
                 order=['Yes', 'No'], palette='Set1')

# Since it's not used, first ax is removed

axes[0, 0].remove()

plt.show()

Together, kdeplots and histplots can help to understand the conditional distribution of numerical features to target. While kdeplots take in account the difference in the frequency distribution of the target (this is, the line for the target category more frequent is higher), violinplots only show the relative distribution. Customers are more likely to churn if:
- They have enrolled to the company recently (low tenure).
- Their monthly charge is high.
- They don't have any dependent. 
- They haven't given any referral.
- They have more than 65.
- Customers with low average GB download are very unlikely to churn.

Since features that indicate total amounts can be understood as the product of the tenure and some monthly charge, they aren't taken in account for this analysis.

## **Bivariate distribution between categorical features and target**

The same approach can be taken to analyze the relation between categorical features and the target, but the plots recquired are different.

In [ ]:
cols_cat = cols_cat_demographics + cols_cat_services

fig, axes = plt.subplots(nrows=(len(cols_cat) + 3) // 3, ncols=3,
                         figsize=(18, ((len(cols_cat) + 3) // 3) * 4))

plt.suptitle('df_status: bivariate distributions between categorical features and target', 
             y=0.92)

# Countplots in axes layout

sns.countplot(data=df, x='churn_label', ax=axes[0, 0], 
              order=['Yes', 'No'], palette='Set1')

for i, c in enumerate(cols_cat):
  sns.countplot(data=df, x=c, hue='churn_label', ax=axes[(i + 1) // 3, (i + 1) % 3], 
                order=df[c].unique().sort_values(), hue_order=['Yes', 'No'], 
                palette='Set1')

# Axes that are not used are removed

axes[4, 1].remove()
axes[4, 2].remove()

plt.show()

Customers are more likely to churn if:
- They have a month-to-month contract.
- They have paperless billing.
- They pay by bank withdrawal or mailed check.
- They have internet service.
- They do not have any dependents.
- They haven't referred any friend.
- They aren't married.
- They are senior citizens.

Features 'gender', 'offer', 'phone_service' and 'refunds' do not seem to have important effects on churning.


## **Bivariate distribution between sub-services and target**

The approach here is the same, but taking in account only the customers that have subscribed the broader service. In the case of 'phone_service':

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 5))

plt.suptitle('df_status: bivariate distribution between phone sub-service and target', 
             y=0.95)

data = df.query("phone_service=='Yes'")

# Countplots in axes layout

sns.countplot(data=data, x='churn_label', ax=axes[0], 
              order=['Yes', 'No'], palette='Set1')

sns.countplot(data=data, x='multiple_lines', hue='churn_label', ax=axes[1], 
                order=data['multiple_lines'].unique().sort_values(), hue_order=['Yes', 'No'], 
                palette='Set1')

plt.show()

Customers that have phone service are more likely to churn if they have multiple lines. It was said before that 'phone_service' does not have important effects on churning, but it must be considered together with 'multiple_lines' to identify those customers that have only one line from those who don't have phone service. Because of this, it makes sense to join them in one feature.

In [ ]:
cond_list = [(df['phone_service'] == 'No'), 
             (df['phone_service'] == 'Yes') & 
             (df['multiple_lines'] == 'No'), 
             (df['multiple_lines'] == 'Yes')]

choice_list = ['No', 'One_Line', 'Multiple_Lines']

df['phone_service'] = np.select(cond_list, choice_list)

df['phone_service'] = pd.Categorical(df['phone_service'])

df.drop(columns='multiple_lines', inplace=True)

In the case of 'internet_services':

In [ ]:
fig, axes = plt.subplots(nrows=(len(cols_cat_internet_services) + 3) // 3, ncols=3,
                         figsize=(18, ((len(cols_cat_internet_services) + 3) // 3) * 4))

plt.suptitle('df_status: bivariate distributions between internet sub-services and target', 
             y=0.92)

data = df.query("internet_service=='Yes'")

# Countplots in axes layout

sns.countplot(data=data, x='churn_label', ax=axes[0, 0], 
              order=['Yes', 'No'], palette='Set1')

for i, c in enumerate(cols_cat_internet_services):
  sns.countplot(data=data, x=c, hue='churn_label', ax=axes[(i + 1) // 3, (i + 1) % 3], 
                order=data[c].unique().sort_values(), hue_order=['Yes', 'No'], 
                palette='Set1')

# Axes that are not used are removed

axes[3, 1].remove()
axes[3, 2].remove()

plt.show()

Customers that have internet service are more likely to churn if:
- The type of their service is Fiber Optic.
- They don't have online back-up.
- They don't have online security.
- They don't have device protection plan.
- They don't have premium tech support.

Features 'unlimited_data' and those related with streaming do not seem to have important effects on churning.

Since in this case there are several sub-services, any attempt to construct new features with the combination of cases would increase cardinality. The best choice here seems to be drop 'internet_service', because its 'No' cases are completely overlapped with 'None' cases in 'internet_type'. Despite that for the other internet sub-services, cases in which the customer doesn't have internet service would be mixed with cases in which the customer has internet service but doesn't have the sub-service, an appropriate model that consideres cross effects could fix these issues.

## **Multiple correspondence analysis**

Since the target is categorical, a multiple correspondence analysis can help to find the categories more related to churning. In order to see the effects better and to capture more inertia, only those features than were founded to be related with target are included.

In [ ]:
cols_mca = ['married', 'dependents', 'age_category', 'referred_a_friend', 
            'contract', 'paperless_billing', 'payment_method', 
            'phone_service', 'internet_type', 
            'online_security', 'online_backup', 'device_protection_plan', 
            'churn_label']

mca = MCA(n_components=len(cols_mca))
mca = mca.fit(df[cols_mca])

ax = mca.plot_coordinates(
     X = df[cols_mca],
     ax=None,
     figsize=(16, 16),
     show_row_points=False,
     row_points_size=7,
     show_row_labels=False,
     show_column_points=True,
     column_points_size=40,
     show_column_labels=True,
     legend_n_cols=1
     )

## **Multivariate distribution between target, categorical and numerical features**

It's possible to go further and study multivariate distribution between the target and groups of numerical and categorical features. Despite it's not realistic to explore all the combinations, it's still useful to do this at least with features that the bivariate analysis has proved to be related with the target.

In [ ]:
sns.relplot(data=df.sample(1000), x='tenure_in_months', y='monthly_charge', 
            row= 'payment_method', col='contract', hue='churn_label', 
            hue_order=['Yes', 'No'], kind='scatter', size=3, palette='Set1')

plt.suptitle("df: multivariate distribution of some features and target", y=1.05)

plt.show()

It's clear from the plots that as 'monthly_charge' gets higher and 'tenure_in_months' gets lower, churning is more likely. It's especially true for month-to-month contracts, and for bank withdrawals and mailed check payment methods.

In [ ]:
sns.catplot(data=df.sample(150), x='satisfaction_score', y='dependents', 
            row='internet_type', col='phone_service', hue='churn_label', 
            hue_order=['Yes', 'No'], kind='swarm', palette='Set1')

plt.suptitle("df: multivariate distribution of some features and target", y=1.02)

plt.show()

Similarly, churning is strongly associated with low levels of 'satisfaction_score', especially for Fiber Optic internet service and multiple phone lines.

## **Conclusion of Data Curation**

After making a carefully inspection of the relation between the target and the features, the following ones were found to be more relevant:
- 'satisfaction_score'
- 'tenure_in_months'
- 'monthly_charge'
- 'avg_monthly_gb_download'
- 'contract'
- 'payment_method' (dichotomizing depending whether the customer pays with credit card or not)
- 'paperless_billing'
- 'dependents'
- 'referred_a_friend'
- 'married'
- 'age_category' (dichotomizing depending whether the customer is senior citizen or not)
- 'phone_service' (including the 'multiple_lines' sub-service inside this feature)
- 'internet_type'
- 'online_backup'
- 'online_security'
- 'device_protection_plan'
- 'premium_tech_support'

Feature 'churn_score' is not included because it's the output of the previous model. Instead, it could be used as a baseline to compare new models.

Additionally, the analysis made in the last section can enrich the conclusions of the EDA. It can be noted that customers that suscribed to internet sub-services are more likely to churn, with the exception of unlimited data and streaming sub-services. Particularly, the company does not charge any additional fee for streaming (but probably, there are fees charged by the third party provider). This could be an indicator of a bad relation between the quality of premium services and its pricing or, as the geographical analysis suggested, just that in big cities and counties the competence is doing better. Its expectable that the population in that zones has more options to suscribe, and expects better offers at lower prices.